# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f74d477bf10>
├── 'a' --> tensor([[ 0.2521, -2.1569, -0.6048],
│                   [-0.1872,  1.4476, -0.4937]])
└── 'x' --> <FastTreeValue 0x7f74d47192b0>
    └── 'c' --> tensor([[-1.3475,  0.9497, -0.2398, -0.7133],
                        [-0.0330, -1.7767, -0.1610, -1.4573],
                        [-0.5588,  1.0257,  0.2022, -0.7223]])

In [4]:
t.a

tensor([[ 0.2521, -2.1569, -0.6048],
        [-0.1872,  1.4476, -0.4937]])

In [5]:
%timeit t.a

66.2 ns ± 0.0342 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f74d477bf10>
├── 'a' --> tensor([[-0.9307, -0.3243, -0.9877],
│                   [-1.7647,  1.8510,  1.0412]])
└── 'x' --> <FastTreeValue 0x7f74d47192b0>
    └── 'c' --> tensor([[-1.3475,  0.9497, -0.2398, -0.7133],
                        [-0.0330, -1.7767, -0.1610, -1.4573],
                        [-0.5588,  1.0257,  0.2022, -0.7223]])

In [7]:
%timeit t.a = new_value

66.3 ns ± 0.0465 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2521, -2.1569, -0.6048],
               [-0.1872,  1.4476, -0.4937]]),
    x: Batch(
           c: tensor([[-1.3475,  0.9497, -0.2398, -0.7133],
                      [-0.0330, -1.7767, -0.1610, -1.4573],
                      [-0.5588,  1.0257,  0.2022, -0.7223]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2521, -2.1569, -0.6048],
        [-0.1872,  1.4476, -0.4937]])

In [11]:
%timeit b.a

58.9 ns ± 0.0893 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.6097, -0.4221,  0.6210],
               [ 0.6138,  0.9978,  0.1225]]),
    x: Batch(
           c: tensor([[-1.3475,  0.9497, -0.2398, -0.7133],
                      [-0.0330, -1.7767, -0.1610, -1.4573],
                      [-0.5588,  1.0257,  0.2022, -0.7223]]),
       ),
)

In [13]:
%timeit b.a = new_value

486 ns ± 0.637 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

893 ns ± 9.04 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 33 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 580 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 553 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f74d478d520>
├── 'a' --> tensor([[[ 0.2521, -2.1569, -0.6048],
│                    [-0.1872,  1.4476, -0.4937]],
│           
│                   [[ 0.2521, -2.1569, -0.6048],
│                    [-0.1872,  1.4476, -0.4937]],
│           
│                   [[ 0.2521, -2.1569, -0.6048],
│                    [-0.1872,  1.4476, -0.4937]],
│           
│                   [[ 0.2521, -2.1569, -0.6048],
│                    [-0.1872,  1.4476, -0.4937]],
│           
│                   [[ 0.2521, -2.1569, -0.6048],
│                    [-0.1872,  1.4476, -0.4937]],
│           
│                   [[ 0.2521, -2.1569, -0.6048],
│                    [-0.1872,  1.4476, -0.4937]],
│           
│                   [[ 0.2521, -2.1569, -0.6048],
│                    [-0.1872,  1.4476, -0.4937]],
│           
│                   [[ 0.2521, -2.1569, -0.6048],
│                    [-0.1872,  1.4476, -0.4937]]])
└── 'x' --> <FastTreeValue 0x7f7428b50df0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.9 µs ± 80.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f74283da850>
├── 'a' --> tensor([[ 0.2521, -2.1569, -0.6048],
│                   [-0.1872,  1.4476, -0.4937],
│                   [ 0.2521, -2.1569, -0.6048],
│                   [-0.1872,  1.4476, -0.4937],
│                   [ 0.2521, -2.1569, -0.6048],
│                   [-0.1872,  1.4476, -0.4937],
│                   [ 0.2521, -2.1569, -0.6048],
│                   [-0.1872,  1.4476, -0.4937],
│                   [ 0.2521, -2.1569, -0.6048],
│                   [-0.1872,  1.4476, -0.4937],
│                   [ 0.2521, -2.1569, -0.6048],
│                   [-0.1872,  1.4476, -0.4937],
│                   [ 0.2521, -2.1569, -0.6048],
│                   [-0.1872,  1.4476, -0.4937],
│                   [ 0.2521, -2.1569, -0.6048],
│                   [-0.1872,  1.4476, -0.4937]])
└── 'x' --> <FastTreeValue 0x7f74284432e0>
    └── 'c' --> tensor([[-1.3475,  0.9497, -0.2398, -0.7133],
                        [-0.0330, -1.7767, -0.1610, -1.4573],
                 

In [23]:
%timeit t_cat(trees)

30.7 µs ± 185 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.3 µs ± 402 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2521, -2.1569, -0.6048],
                [-0.1872,  1.4476, -0.4937]],
       
               [[ 0.2521, -2.1569, -0.6048],
                [-0.1872,  1.4476, -0.4937]],
       
               [[ 0.2521, -2.1569, -0.6048],
                [-0.1872,  1.4476, -0.4937]],
       
               [[ 0.2521, -2.1569, -0.6048],
                [-0.1872,  1.4476, -0.4937]],
       
               [[ 0.2521, -2.1569, -0.6048],
                [-0.1872,  1.4476, -0.4937]],
       
               [[ 0.2521, -2.1569, -0.6048],
                [-0.1872,  1.4476, -0.4937]],
       
               [[ 0.2521, -2.1569, -0.6048],
                [-0.1872,  1.4476, -0.4937]],
       
               [[ 0.2521, -2.1569, -0.6048],
                [-0.1872,  1.4476, -0.4937]]]),
    x: Batch(
           c: tensor([[[-1.3475,  0.9497, -0.2398, -0.7133],
                       [-0.0330, -1.7767, -0.1610, -1.4573],
                       [-0.5588,  1.0257,  0.2022, -0.7223]],
         

In [26]:
%timeit Batch.stack(batches)

78.1 µs ± 211 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2521, -2.1569, -0.6048],
               [-0.1872,  1.4476, -0.4937],
               [ 0.2521, -2.1569, -0.6048],
               [-0.1872,  1.4476, -0.4937],
               [ 0.2521, -2.1569, -0.6048],
               [-0.1872,  1.4476, -0.4937],
               [ 0.2521, -2.1569, -0.6048],
               [-0.1872,  1.4476, -0.4937],
               [ 0.2521, -2.1569, -0.6048],
               [-0.1872,  1.4476, -0.4937],
               [ 0.2521, -2.1569, -0.6048],
               [-0.1872,  1.4476, -0.4937],
               [ 0.2521, -2.1569, -0.6048],
               [-0.1872,  1.4476, -0.4937],
               [ 0.2521, -2.1569, -0.6048],
               [-0.1872,  1.4476, -0.4937]]),
    x: Batch(
           c: tensor([[-1.3475,  0.9497, -0.2398, -0.7133],
                      [-0.0330, -1.7767, -0.1610, -1.4573],
                      [-0.5588,  1.0257,  0.2022, -0.7223],
                      [-1.3475,  0.9497, -0.2398, -0.7133],
                      [-0.0330, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 322 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 745 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
